# CategoricalGrouping Notebook - AAAG

In [1]:
# Basics Importation 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

# For Detailed Stats Output
import statsmodels.api as sm
# The linear regression models 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV, ElasticNetCV
# Instantiating the linear regression models
ols = LinearRegression()
ridge = Ridge()
lasso  = Lasso()
lassocv = LassoCV()
elasticnet = ElasticNet()
elasticnetcv = ElasticNetCV()
# The tree model
from sklearn import tree
# Instantiating the tree model (regression type)
regressor = tree.DecisionTreeRegressor()
# The ensemble model for random forest and bagging
from sklearn import ensemble
# Instantiating the ensemble models
randomForest = ensemble.RandomForestRegressor()
bagging = ensemble.BaggingRegressor()
# Instantiating the boost models
gbm = ensemble.GradientBoostingRegressor()
abr = ensemble.AdaBoostRegressor()
# xg boost
import xgboost as xgb
# lg boost
import lightgbm as lgb
# K mean clustering
from sklearn.cluster import KMeans
kmeans = KMeans()
# The model selection for cross validation, k fold splits, train_test_split, grid search etc. 
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# Some automatic feature selection functions (recursive finding, best feature selection etc.)
import sklearn.feature_selection as fs
# Importing the different error evaluation/metrics
from sklearn import metrics
from sklearn.metrics import silhouette_score
# Importing PCA
from sklearn.decomposition import PCA
# Making it so that we can see all columns of the dataframe
pd.set_option('display.max_columns', None)

# Import data
standardized_no_outlier_undummified_df_total=pd.read_csv('standardized_no_outlier_undummified_df_total.csv',index_col='Id')
standardized_with_outlier_undummified_df_test=pd.read_csv('standardized_with_outlier_undummified_df_test.csv',index_col='Id')
# Specify df to work with:
df = standardized_no_outlier_undummified_df_total
df_test = standardized_with_outlier_undummified_df_test
df_test.shape
# df_test must have : undummfieid: (1459, 70), dummified: (1459,902)

(1459, 70)

In [2]:
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass',\
             'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MoSold']
ordinal_var_processed=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath',\
             'HalfBath','BedroomAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond']
cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','GrLivArea','GarageYrBlt','GarageArea']
cont_var_for_tuning=ordinal_var_processed+cont_var_processed

In [3]:
x=df#.drop(['SalePrice'],axis='columns')
y=df.OverallQual
x_test=df_test#.drop('OverallQual',axis='columns')
y_test=df_test.OverallQual

In [4]:
def dummify_column(dataframe,column_name):
    '''
    ### NOTE!!! ### vector operation not working yet!!!! ###
    - dataframe takes the entire dataframe you are working on
    - column_name takes a list of strings, where the strings are the column names
    '''
    for feature in column_name:
        dummified_feature = pd.get_dummies(dataframe.loc[:,feature], prefix=feature, prefix_sep='__',drop_first=True)
        dummified_df = pd.concat([dataframe.drop(feature,axis=1),dummified_feature],axis=1,sort='False')
    return dummified_df

In [5]:
# TAKES A LONG TIME TO RUN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# TAKES A LONG TIME TO RUN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# K mean clustering performed on categorical variables
# perform for train (non-test) set
nominal_var_cluster_num=[]
nominal_var_cluster_num_test=[]
for cat_var in nominal_var_processed:
    # dummifying the nominal categorical variable
    dummified_column=pd.get_dummies(x.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster=pd.concat([y,dummified_column],axis=1,sort='False')
    print(cat_var)
    # Finding the optimal number of clusters and storing into nominal_var_cluster_num
    kmax = df_for_cluster.shape[1]
    KNumberChoice=range(2, kmax)
    # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
    # Doing mulitple trials for each category
    cluster_count=[]
    for trial_num in range(0,5):
#         print(trial_num)
        sil = []
        for k in range(2, kmax):
            kmeans = KMeans(n_clusters = k,random_state=trial_num,init='k-means++').fit(df_for_cluster)
            labels = kmeans.labels_
            sil.append(silhouette_score(df_for_cluster, labels, metric = 'euclidean'))
        if np.argmax(sil)<0.2: # defining threshold for 1 cluster
            cluster_count.append(1)
        else:
            cluster_count.append(KNumberChoice[np.argmax(sil)])
    nominal_var_cluster_num.append(max(set(cluster_count), key=cluster_count.count))

    # perform for the test set #######################################       
    dummified_column_test=pd.get_dummies(x_test.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster_test=pd.concat([y_test,dummified_column_test],axis=1,sort='False')
    kmax_test = df_for_cluster_test.shape[1]
    KNumberChoice_test=range(2, kmax_test)
    cluster_count_test=[]
    for trial_num in range(0,5):
#         print(trial_num)
        sil = []
        for k in range(2, kmax_test):
            kmeans_test = KMeans(n_clusters = k,random_state=trial_num,init='k-means++').fit(df_for_cluster_test)
            labels_test = kmeans_test.labels_
            sil.append(silhouette_score(df_for_cluster_test, labels_test, metric = 'euclidean'))
        if np.argmax(sil)<0.2: # defining threshold for 1 cluster
            cluster_count_test.append(1)
        else:
            cluster_count_test.append(KNumberChoice_test[np.argmax(sil)])
    nominal_var_cluster_num_test.append(max(set(cluster_count_test), key=cluster_count_test.count))
print(nominal_var_cluster_num)
print(nominal_var_cluster_num_test)

MSZoning
Street
Alley
LotShape
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
Foundation
BsmtFinType1
Heating
CentralAir
Electrical
Functional
GarageType
GarageFinish
PavedDrive
Fence
MiscFeature
SaleType
SaleCondition
MSSubClass
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MoSold
[5, 1, 3, 4, 4, 5, 3, 25, 9, 6, 5, 8, 6, 6, 14, 15, 5, 6, 7, 6, 1, 5, 7, 7, 4, 3, 5, 1, 9, 6, 16, 1, 1, 1, 1, 1, 1, 12]
[5, 1, 3, 4, 4, 5, 3, 25, 9, 5, 5, 7, 6, 4, 13, 15, 5, 6, 7, 4, 1, 4, 7, 7, 4, 3, 5, 4, 9, 6, 16, 1, 1, 1, 1, 1, 1, 12]


In [19]:
# With the known cluster number for each categorical variable, perform the clustering
# perform for train (non-test) set
for ind,cat_var in enumerate(nominal_var_processed):
    dummified_column=pd.get_dummies(x.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster=pd.concat([y,dummified_column],axis=1,sort='False')
    kmeans=KMeans(n_clusters=nominal_var_cluster_num[ind]).fit(df_for_cluster)
    x.loc[:,cat_var]=kmeans.labels_
    
    # perform for the test set###################
    dummified_column_test=pd.get_dummies(x_test.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster_test=pd.concat([y_test,dummified_column_test],axis=1,sort='False')
    kmeans_test=KMeans(n_clusters=nominal_var_cluster_num_test[ind]).fit(df_for_cluster_test)
    x_test.loc[:,cat_var]=kmeans_test.labels_
    
clustered_df=pd.concat([y,x],axis=1,sort='False')    
clustered_df_test=pd.concat([y_test,x_test],axis=1,sort='False')    
print(clustered_df.shape)
print(clustered_df_test.shape)

(2399, 72)
(1459, 71)


In [21]:
# Removing the columns with pure 0's
undummified_clustered_df=clustered_df.loc[:,(clustered_df != 0).any(axis=0)]
undummified_clustered_df.shape

(2399, 63)

In [22]:
cat_cols_to_keep=set(undummified_clustered_df.columns)-set(cont_var_for_tuning)-{'SalePrice'}
# dummifying the clustered_df
undummified_clustered_df_copy=undummified_clustered_df.copy()
for feature in cat_cols_to_keep:
    undummified_clustered_df_copy=dummify_column(undummified_clustered_df_copy,[feature])
dummified_clustered_df=undummified_clustered_df_copy
print(dummified_clustered_df.isnull().sum().sum())
print(dummified_clustered_df.shape)
# dummifying the clustered_df_test
undummified_clustered_df_test_copy=clustered_df_test.copy()
for feature in cat_cols_to_keep:
    undummified_clustered_df_test_copy=dummify_column(undummified_clustered_df_test_copy,[feature])
dummified_clustered_df_test=undummified_clustered_df_test_copy
print(dummified_clustered_df_test.isnull().sum().sum())
print(dummified_clustered_df_test.shape)

1209
(2399, 221)
0
(1459, 221)


In [23]:
dummified_clustered_df_test.head()

,OverallQual,LotFrontage,LotArea,Street,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,TotalBsmtSF,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscFeature,YrSold,LandContour__1,LandContour__2,LandContour__3,MSSubClass__1,MSSubClass__2,MSSubClass__3,MSSubClass__4,MSSubClass__5,MSSubClass__6,MSSubClass__7,MSSubClass__8,MSSubClass__9,MSSubClass__10,MSSubClass__11,MSSubClass__12,MSSubClass__13,MSSubClass__14,MSSubClass__15,MSZoning__1,MSZoning__2,MSZoning__3,MSZoning__4,Exterior2nd__1,Exterior2nd__2,Exterior2nd__3,Exterior2nd__4,Exterior2nd__5,Exterior2nd__6,Exterior2nd__7,Exterior2nd__8,Exterior2nd__9,Exterior2nd__10,Exterior2nd__11,Exterior2nd__12,Exterior2nd__13,Exterior2nd__14,PavedDrive__1,PavedDrive__2,SaleType__1,SaleType__2,SaleType__3,SaleType__4,SaleType__5,SaleType__6,SaleType__7,SaleType__8,BldgType__1,BldgType__2,BldgType__3,BldgType__4,SaleCondition__1,SaleCondition__2,SaleCondition__3,SaleCondition__4,SaleCondition__5,RoofMatl__1,RoofMatl__2,RoofMatl__3,Fence__1,Fence__2,Fence__3,Fence__4,Functional__1,Functional__2,Functional__3,Functional__4,Functional__5,Functional__6,LandSlope__1,LandSlope__2,Foundation__1,Foundation__2,Foundation__3,Foundation__4,Foundation__5,LotConfig__1,LotConfig__2,LotConfig__3,LotConfig__4,GarageType__1,GarageType__2,GarageType__3,GarageType__4,GarageType__5,GarageType__6,GarageFinish__1,GarageFinish__2,GarageFinish__3,LotShape__1,LotShape__2,LotShape__3,Condition1__1,Condition1__2,Condition1__3,Condition1__4,Condition1__5,Condition1__6,Condition1__7,Condition1__8,MoSold__1,MoSold__2,MoSold__3,MoSold__4,MoSold__5,MoSold__6,MoSold__7,MoSold__8,MoSold__9,MoSold__10,MoSold__11,HouseStyle__1,HouseStyle__2,HouseStyle__3,HouseStyle__4,HouseStyle__5,HouseStyle__6,Alley__1,Alley__2,Exterior1st__1,Exterior1st__2,Exterior1st__3,Exterior1st__4,Exterior1st__5,Exterior1st__6,Exterior1st__7,Exterior1st__8,Exterior1st__9,Exterior1st__10,Exterior1st__11,Exterior1st__12,Electrical__1,Electrical__2,Electrical__3,Condition2__1,Condition2__2,Condition2__3,Condition2__4,Neighborhood__1,Neighborhood__2,Neighborhood__3,Neighborhood__4,Neighborhood__5,Neighborhood__6,Neighborhood__7,Neighborhood__8,Neighborhood__9,Neighborhood__10,Neighborhood__11,Neighborhood__12,Neighborhood__13,Neighborhood__14,Neighborhood__15,Neighborhood__16,Neighborhood__17,Neighborhood__18,Neighborhood__19,Neighborhood__20,Neighborhood__21,Neighborhood__22,Neighborhood__23,Neighborhood__24,RoofStyle__1,RoofStyle__2,RoofStyle__3,RoofStyle__4,RoofStyle__5,BsmtFinType1__1,BsmtFinType1__2,BsmtFinType1__3,BsmtFinType1__4,BsmtFinType1__5,BsmtFinType1__6,MasVnrType__1,MasVnrType__2,MasVnrType__3,MasVnrType__4,Heating__1,Heating__2,Heating__3
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,-0.751101,0.526150,0.363929,0,-0.751101,0.400766,-0.340945,-1.072885,-0.563316,-0.678102,-0.223518,-0.499340,0.164335,-0.575738,-0.368484,-1.211189,0,-0.654561,-0.775254,-1.215588,-0.818068,-0.25816,-1.028720,-0.751040,-1.029543,-0.761697,-0.918335,-0.898055,-0.950143,-0.650488,-0.984704,1.185511,0.294032,0.271153,0,0,0,0,0,0,1,1.713905,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
1462,-0.054877,0.573788,0.897861,0,-0.054877,0.400766,-0.439695,-1.214908,0.047057,-0.678102,-0.223518,-0.4

In [24]:
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Removing sub-categories with low counts and their associated observations (if less than 2% of total observation)
boolmatrix=[]
non_cont_columns=dummified_clustered_df.columns[~dummified_clustered_df.columns.isin(cont_var_for_tuning)]
for cat_feature in non_cont_columns:
    if dummified_clustered_df.loc[:,cat_feature].sum()<0.020*dummified_clustered_df.shape[0]:
        print('yes')
        boolvec=(dummified_clustered_df.loc[:,cat_feature]==1)
        boolmatrix.append(list(boolvec))
   

yes
yes
yes
yes
yes
yes
yes
yes
yes


In [25]:
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# creating boolean vector to takeout observations that have categorical low count observations
if boolmatrix!=[]:
    reduction_bool_vec=np.any(np.array(boolmatrix).transpose(),axis=1)
    temp_df=dummified_clustered_df[~reduction_bool_vec]
    purged_dummified_clustered_df=temp_df.loc[:,(temp_df != 0).any(axis=0)]
else:
    purged_dummified_clustered_df=dummified_clustered_df
print(purged_dummified_clustered_df.shape)
print(dummified_clustered_df.shape)

(2056, 212)
(2399, 221)


In [26]:
# Removing from the test set columns there were removed from the training set!
cols_to_keep=set(dummified_clustered_df_test.columns)&(set(purged_dummified_clustered_df.columns))


purged_dummified_clustered_df_test=dummified_clustered_df_test.loc[:,list(cols_to_keep)]
purged_dummified_clustered_df=dummified_clustered_df.loc[:,list(cols_to_keep)+['SalePrice']]

# purged_dummified_clustered_df.reset_index(inplace=True)
# purged_dummified_clustered_df.set_index('Id')
# purged_dummified_clustered_df_test.reset_index(inplace=True)
# purged_dummified_clustered_df_test.set_index('Id')
# purged_dummified_clustered_df

purged_dummified_clustered_df.sample(20)

,Fence__4,Exterior1st__9,LandSlope__1,KitchenQual,MoSold__4,Heating__2,RoofStyle__2,Neighborhood__9,LandContour__1,MSSubClass__3,HouseStyle__3,RoofMatl__2,GarageFinish__3,SaleType__1,HouseStyle__2,ExterCond,Fence__1,GarageCond,HalfBath,MoSold__8,Functional__3,Neighborhood__18,GrLivArea,Exterior1st__2,Condition1__1,Neighborhood__6,MSSubClass__10,Exterior2nd__3,Fence__3,BsmtFinType1__4,Exterior1st__4,Exterior2nd__4,GarageType__4,LotShape__2,PavedDrive__1,Functional__2,Alley__2,MSSubClass__6,PavedDrive__2,Exterior2nd__9,RoofStyle__5,HouseStyle__5,BsmtFinType1__5,Exterior2nd__11,Exterior1st__7,BsmtFinType1__1,Alley__1,MSSubClass__5,MSSubClass__14,Neighborhood__13,Exterior2nd__8,Condition1__4,Exterior2nd__10,MoSold__10,MSSubClass__8,OverallQual,OverallQual,GarageType__5,MSSubClass__11,MoSold__2,MSSubClass__4,Exterior2nd__7,TotalBsmtSF,LotShape__1,MoSold__3,MoSold__7,Neighborhood__2,Functional__5,Neighborhood__10,SaleType__5,Electrical__1,Neighborhood__16,SaleType__8,SaleType__7,Functional__4,Exterior1st__1,Exterior2nd__2,Neighborhood__14,BedroomAbvGr,2ndFlrSF,GarageFinish__1,FireplaceQu,RoofStyle__1,Condition2__3,Electrical__3,RoofStyle__3,SaleType__3,Exterior2nd__14,Foundation__4,Neighborhood__5,HouseStyle__4,MSZoning__3,Exterior1st__6,MSSubClass__13,MoSold__1,BsmtFinType1__3,SaleCondition__2,Exterior1st__8,Condition1__7,LotShape__3,LotArea,Condition1__3,BsmtFinType1__2,Neighborhood__15,Exterior2nd__6,BldgType__4,Neighborhood__23,LotFrontage,Neighborhood__11,MasVnrArea,SaleCondition__5,GarageType__2,Condition1__2,MasVnrType__3,Neighborhood__19,LotConfig__4,MSSubClass__15,Neighborhood__1,GarageQual,BsmtFinType1__6,BldgType__1,OverallCond,Exterior1st__5,Neighborhood__21,LotConfig__2,Exterior2nd__1,Condition1__6,HouseStyle__6,BsmtHalfBath,ExterQual,FullBath,Foundation__3,SaleCondition__4,GarageType__6,HouseStyle__1,Exterior1st__3,SaleType__2,Condition2__2,Condition2__4,MasVnrType__1,Exterior2nd__12,GarageType__3,MSSubClass__12,LotConfig__1,Neighborhood__20,SaleType__6,Exterior2nd__5,Heating__3,GarageFinish__2,MoSold__9,YearBuilt,TotRmsAbvGrd,BsmtFullBath,RoofMatl__3,MoSold__11,MasVnrType__2,Neighborhood__8,BsmtExposure,LotConfig__3,MSSubClass__2,Fence__2,Functional__1,Exterior1st__11,LandContour__2,MoSold__6,GarageCars,Neighborhood__12,MSZoning__2,GarageType__1,Fireplaces,BldgType__2,Neighborhood__3,GarageArea,MSZoning__4,Condition1__5,SaleCondition__1,YrSold,Heating__1,HeatingQC,MSSubClass__7,Foundation__1,LandContour__3,MoSold__5,RoofMatl__1,Electrical__2,MSZoning__1,Foundation__2,BldgType__3,SaleCondition__3,1stFlrSF,Foundation__5,Exterior1st__12,BsmtQual,Condition2__1,SaleType__4,RoofStyle__4,MasVnrType__4,MSSubClass__1,LandSlope__2,Exterior1st__10,BsmtCond,GarageYrBlt,YearRemodAdd,MSSubClass__9,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2459,1,0,1,-0.691857,0,0,0,0,1,0,0,0,0,0,0,-0.230717,0,0.288172,-0.704142,0,0,0,-1.344802,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-0.683681,-0.683681,0,0,0,0,0,-0.230305,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.035821,-0.789605,0,-0.850376,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-0.796165,0,1,0,0,0,0,-0.844772,0,-0.558041,0,0,0,0,0,0,0,0,0.304728,0,0,1.233343,0,0,0,0,0,0,-0.259376,1.341540,-0.926353,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,-1.008135,-0.856792,-0.783794,0,0,1,0,-0.529942,0,0,0,0,0,0,0,-0.902239,0,0,1,-0.830698,0,0,-1.021897,0,0,1,-0.619841,1,0.943895,0,0,0,0,0,0,0,0,0,0,-0.609217,1,0,-0.448765,0,0,0,0,0,0,0,0.150978,-1.493233,-1.546506,0,NaN
1275,0,0,1,-0.691857,0,0,0,0,1,0,0,0,0,0,0,-0.230717,1,0.288172,-0.704142,0,0,0,-0.301493,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.683681,-0.683681,0,0,0,0,0,-0.877220,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.333289,0.808095,0,1.445041,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1.010326,0,1,0,0,0,0,-0.723759,0,-0.558041,0,0,0,0,0,1,0,0,0.304728,0,0,0.3

In [27]:
purged_dummified_clustered_df.to_csv('purged_dummified_clustered_df.csv')
purged_dummified_clustered_df_test.to_csv('purged_dummified_clustered_df_test.csv')

In [28]:
def undummify(dataframe):
    tot_col=dataframe.columns
    cat_col=list(tot_col[tot_col.str.contains('__')])
    cat_col_split=set(map(lambda x:x.split('__')[0],cat_col))
    cat_dict={}
    for col in cat_col_split:
        sub_df=dataframe[cat_col].loc[:,list(map(lambda x:col in x, dataframe[cat_col].columns))]
        for i in sub_df.columns:
            label_num=int(i.split('__')[1])
            sub_df.loc[:,i]=np.array(sub_df.loc[:,i])*label_num
        cat_dict[col]=sub_df.sum(axis=1)+1
    df1=dataframe.drop(cat_col,axis=1)
    df2=pd.DataFrame(cat_dict)
    return pd.concat([df1,df2],axis=1)

In [29]:
a=undummify(purged_dummified_clustered_df)
a.head(15)

,KitchenQual,ExterCond,GarageCond,HalfBath,GrLivArea,OverallQual,OverallQual,TotalBsmtSF,BedroomAbvGr,2ndFlrSF,FireplaceQu,LotArea,LotFrontage,MasVnrArea,GarageQual,OverallCond,BsmtHalfBath,ExterQual,FullBath,YearBuilt,TotRmsAbvGrd,BsmtFullBath,BsmtExposure,GarageCars,Fireplaces,GarageArea,YrSold,HeatingQC,1stFlrSF,BsmtQual,BsmtCond,GarageYrBlt,YearRemodAdd,SalePrice,MSZoning,Exterior2nd,BldgType,RoofMatl,LandSlope,LotConfig,GarageType,LotShape,MoSold,Exterior1st,Alley,Electrical,Condition2,LandContour,MSSubClass,SaleType,PavedDrive,SaleCondition,Fence,Functional,Foundation,GarageFinish,Condition1,Neighborhood,HouseStyle,RoofStyle,BsmtFinType1,MasVnrType,Heating
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.961295,-0.230717,0.288172,1.340939,1.009770,0.924310,0.924310,-0.311532,0.333289,1.526925,-0.850376,-0.137861,0.002320,1.001902,0.304728,-0.552826,-0.259376,1.341540,0.999402,1.131878,1.492287,1.185130,-0.529942,0.531186,-0.830698,0.628728,0.143897,0.943895,-0.705778,0.725267,0.150978,1.069886,0.962952,12.247699,3,4,2,2,1,3,1,3,2,4,2,2,2,3,5,2,2,5,1,2,2,4,3,21,1,3,2,2,4
2,-0.691857,-0.230717,0.288172,-0.704142,-0.267286,0.120314,0.120314,0.866258,0.333289,-0.789605,0.871187,0.187053,0.909920,-0.558041,0.304728,2.126427,3.703949,-0.612531,0.999402,0.229060,-0.073766,-0.783794,2.516377,0.531186,0.849610,0.157120,-0.619841,0.943895,0.694364,0.725267,0.150978,-0.011430,-0.315451,12.109016,4,7,3,3,3,1,3,3,7,2,3,3,4,1,15,3,1,3,3,3,4,4,6,3,3,1,5,4,1
3,0.961295,-0.230717,0.288172,1.340939,1.226414,0.924310,0.924310,-0.125871,0.333289,1.559476,0.871187,0.653234,0.183840,0.731299,0.304728,-0.552826,-0.259376,1.341540,0.999402,1.065003,-0.073766,1.185130,0.485498,0.531186,0.849610,0.950278,0.143897,0.943895,-0.485066,0.725267,0.150978,0.989789,0.915604,12.317171,3,4,2,2,1,3,1,1,2,4,2,2,2,3,5,2,2,5,1,2,2,4,3,21,1,3,2,2,4
4,0.961295,-0.230717,0.288172,-0.704142,1.029724,0.924310,0.924310,-0.601629,0.333289,1.261093,1.445041,0.172926,-0.300213,-0.558041,0.304728,-0.552826,-0.259376,-0.612531,-0.926353,-1.810640,0.709261,1.185130,-0.529942,1.964611,0.849610,1.132490,-1.383578,-0.099609,-0.343673,-0.448765,1.883977,0.869643,-0.599541,11.849405,3,15,2,2,1,1,6,1,2,9,2,2,2,3,16,2,2,1,1,2,6,2,3,2,1,3,2,4,4
5,0.961295,-0.230717,0.288172,1.340939,2.400849,1.728305,1.728305,0.526846,1.702399,2.066725,0.871187,1.503660,1.151946,2.227571,0.304728,-0.552826,-0.259376,1.341540,0.999402,1.031565,2.275313,1.185130,1.500938,1.964611,0.849610,2.172169,0.143897,0.943895,0.290875,0.725267,0.150978,0.949740,0.820907,12.429220,3,2,2,1,1,3,4,1,5,10,2,2,5,3,5,4,2,5,1,5,2,4,1,15,1,3,6,2,1
6,-0.691857,-0.230717,0.288172,1.340939,0.017771,-0.683681,-0.683681,-0.485590,-2.404931,0.745706,-0.850376,1.462693,1.212453,-0.558041,0.304728,-0.552826,-0.259376,-0.612531,-0.926353,0.797501,-0.856792,1.185130,-0.529942,0.531186,-0.830698,0.264304,0.907635,0.943895,-0.912696,0.725267,0.150978,0.669399,0.584166,11.870607,2,6,1,1,2,2,5,4,1,8,1,1,3,2,2,1,1,2,5,1,1,3,2,16,7,2,1,3,2
7,0.961295,-0.230717,0.288172,-0.704142,0.964161,1.728305,1.728305,2.096267,0.333289,-0.789605,1.445041,0.323799,0.607387,0.922313,0.304728,-0.552826,-0.259376,1.341540,0.999402,1.165316,0.709261,1.185130,1.500938,0.531186,0.849610,1.100335,-0.619841,0.943895,2.184172,1.899300,0.150978,1.109935,1.057648,12.634606,3,2,2,1,1,3,4,3,5,10,2,2,5,3,9,4,2,5,1,5,2,4,1,6,6,3,6,2,1
8,-0.691857,-0.230717,0.288172,1.340939,2.092987,0.924310,0.924310,0.416609,0.333289,1.876846,0.871187,0.407994,0.909920,1.352093,0.304728,0.340258,-0.259376,-0.612531,0.999402,0.128747,0.709261,1.185130,0.485498,0.531186,2.529918,0.285741,0.907635,0.943895,0.159827,0.725267,0.150978,-0.131576,-0.457496,12.206078,3,9,2,2,1,1,1,1,2,3,2,2,2,3,5,2,2,5,1,2,4,4,3,3,1,3,2,2,4
9,-0.691857,-0.230717,0.288172,-0.704142,1.192207,0.924310,0.924310,-0.033040,-1.035821,1.250243,0.871187,-0.796165,-0.844772,-0.558041,-1.024441,-0.552826,-0.259376,-0.612531,0.999402,-1.275637,1.492287,-0.783794,-0.529942,0.531186,2.529918,0.

In [30]:
############# Reducing Continuous/Ordinal Only ################
cont_df=purged_dummified_clustered_df.loc[:,cont_var_for_tuning]
pca = PCA(n_components = 0.90)
pca.fit(cont_df)
reduced_cont_df = pd.DataFrame(pca.transform(cont_df))
# Performing for the test set
cont_df_test=purged_dummified_clustered_df_test.loc[:,cont_var_for_tuning]
pca_test = PCA(n_components = 0.90)
pca_test.fit(cont_df_test)
reduced_cont_df_test = pd.DataFrame(pca_test.transform(cont_df_test))

In [31]:
categorical_purged_dummified_clustered_df=purged_dummified_clustered_df.drop(cont_var_for_tuning,axis=1)
categorical_purged_dummified_clustered_df_test=purged_dummified_clustered_df_test.drop(cont_var_for_tuning,axis=1)

PCA_purged_dummified_clustered_df=pd.concat([categorical_purged_dummified_clustered_df.reset_index(),reduced_cont_df],axis=1)
PCA_purged_dummified_clustered_df_test=pd.concat([categorical_purged_dummified_clustered_df_test.reset_index(),reduced_cont_df_test],axis=1)

In [32]:
PCA_purged_dummified_clustered_df.set_index('Id',inplace=True)
PCA_purged_dummified_clustered_df_test.set_index('Id',inplace=True)
PCA_purged_dummified_clustered_df.to_csv('PCA_purged_dummified_clustered_df.csv')
PCA_purged_dummified_clustered_df_test.to_csv('PCA_dummified_clustered_df_test.csv')

# My Attempt to improve on k mean clustering 
# A Working progress

In [ ]:
# from sklearn.cluster import KMeans
# from sklearn import metrics
# from scipy.spatial.distance import cdist
# import numpy as np
# import matplotlib.pyplot as plt

# # k means determine k
# distortions = []
# kmax = df_for_cluster.shape[1]
# K = range(1,kmax)
# for k in K:
#     kmeanModel = KMeans(n_clusters=k).fit(df_for_cluster)
#     kmeanModel.fit(df_for_cluster)
#     distortions.append(sum(np.min(cdist(df_for_cluster, kmeanModel.cluster_centers_, 'euclidean')+0*(k), axis=1)) / df_for_cluster.shape[0])

# # Plot the elbow
# plt.plot(K, distortions, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()

In [ ]:
# from sklearn.cluster import KMeans
# from sklearn import metrics
# from scipy.spatial.distance import cdist
# import numpy as np
# import matplotlib.pyplot as plt

# # k means determine k
# K = range(1,df_for_cluster.shape[1])
# penalty_term=np.linspace(0.005,0.01,101)
# distortion_matrix = np.zeros([len(K),len(penalty_term)])
# for i,k in enumerate(K):
#     for j,pen_weight in enumerate(penalty_term):
#         #print(i,j)
#         kmeanModel = KMeans(n_clusters=k).fit(df_for_cluster)
#         kmeanModel.fit(df_for_cluster)
#         distortion_matrix[i,j]=sum(np.min(cdist(df_for_cluster, kmeanModel.cluster_centers_, 'euclidean')+pen_weight*(k), axis=1)) / df_for_cluster.shape[0]
# # The minimum index
# np.unravel_index(np.argmin(distortion_matrix, axis=None), distortion_matrix.shape)
# # Plot the elbow
# plt.pcolor(distortion_matrix)
# plt.colorbar();
# plt.xlabel('Penalization');
# plt.ylabel('k clusters');